### **Importing Libraries**

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import wandb
from skimage.feature import hog
from skimage import exposure
import seaborn as sns
import copy
from IPython.display import Image
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import torchvision.models as models
import tqdm
from torchvision import transforms
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

### **Loading Dataset**

In [ ]:
def load_data():
    train_embeddings = torch.load("./Dataset/train_embeddings.pth")
    train_labels = torch.load("./Dataset/train_labels.pth")
    test_embeddings = torch.load("./Dataset/test_embeddings.pth")
    test_labels = torch.load("./Dataset/test_labels.pth")
    text_embeddings = torch.load("./Dataset/text_embedding.pth")
    return train_embeddings, train_labels, test_embeddings, test_labels, text_embeddings

# Visualization using t-SNE
def visualize_tsne(embeddings, labels, title):
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='tab10', alpha=0.7)
    plt.colorbar(scatter, ticks=range(10), label='Classes')
    plt.title(title)
    plt.show()

train_embeddings, train_labels, test_embeddings, test_labels, text_embeddings = load_data()

train_embeddings_cpu = train_embeddings.cpu().numpy()
train_labels_cpu = train_labels.cpu().numpy()
test_embeddings_cpu = test_embeddings.cpu().numpy()
test_labels_cpu = test_labels.cpu().numpy()
text_embeddings_cpu = text_embeddings.cpu().numpy()

visualize_tsne(train_embeddings_cpu, train_labels_cpu, "t-SNE Visualization of Training Embeddings")
visualize_tsne(test_embeddings_cpu, test_labels_cpu, "t-SNE Visualization of Test Embeddings")


### **3.1**

#### **Task-1.1 Classification**

In [ ]:
def euclidean_distance(x1, x2):
    return cdist(x1, x2.reshape(1, -1), metric='euclidean').flatten()

def cosine_distance(x1, x2):
    return cdist(x1, x2.reshape(1, -1), metric='cosine').flatten()

def knn(train_embeddings, train_labels, query, k, metric="euclidean", return_comparisons=False):
    if metric == "euclidean":
        distances = euclidean_distance(train_embeddings, query)
    elif metric == "cosine":
        distances = cosine_distance(train_embeddings, query)
    else:
        raise ValueError("Unsupported distance metric. Use 'euclidean' or 'cosine'.")
    
    comparisons = len(train_embeddings)
    
    nearest_neighbors = np.argsort(distances)[:k]
    
    if return_comparisons:
        return nearest_neighbors, comparisons
    else:
        return nearest_neighbors

def classify(train_embeddings, train_labels, test_embeddings, test_labels, k, metric="euclidean"):
    correct = 0
    total = len(test_labels)
    for i in range(total):
        knn_indices = knn(train_embeddings, train_labels, test_embeddings[i], k, metric)
        predicted_label = np.bincount(train_labels[knn_indices]).argmax()
        correct += (predicted_label == test_labels[i])
    return correct / total

# Evaluate KNN for different k values and distance metrics
k_values = [1, 5, 10]
distance_metrics = ["euclidean", "cosine"]

for k in k_values:
    for metric in distance_metrics:
        accuracy = classify(train_embeddings_cpu, train_labels_cpu, test_embeddings_cpu, test_labels_cpu, k, metric)
        print(f"K={k}, Metric={metric}, Accuracy={accuracy:.4f}")

#### **Task-1.2 Classification**

In [ ]:
def classify_text_embeddings(text_embeddings, test_embeddings, test_labels, k=1, metric="euclidean"):
    correct = 0
    total = len(test_labels)
    
    all_predicted = []
    all_true = []
    
    for i in range(total):
        if metric == "euclidean":
            distances = np.sqrt(np.sum((text_embeddings - test_embeddings[i])**2, axis=1))
        else:  # cosine
            norm_text = np.linalg.norm(text_embeddings, axis=1)
            norm_test = np.linalg.norm(test_embeddings[i])
            distances = 1 - np.dot(text_embeddings, test_embeddings[i]) / (norm_text * norm_test)
        
        knn_indices = np.argsort(distances)[:k]
        predicted_label = knn_indices[0]
        true_label = test_labels[i]
        
        all_predicted.append(predicted_label)
        all_true.append(true_label)
        
        correct += (predicted_label == true_label)
    
    accuracy = correct / total
    
    precision = precision_score(all_true, all_predicted, average='weighted', zero_division=0)
    recall = recall_score(all_true, all_predicted, average='weighted', zero_division=0)
    f1 = f1_score(all_true, all_predicted, average='weighted', zero_division=0)
    
    return accuracy, precision, recall, f1

distance_metrics = ["euclidean", "cosine"]

for metric in distance_metrics:
    accuracy, precision, recall, f1 = classify_text_embeddings(
        text_embeddings_cpu, test_embeddings_cpu, test_labels_cpu, k=1, metric=metric
    )
    print(f"K=1, Metric={metric}, Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1 Score={f1:.4f}")

#### **Task-2.1 Retrieval** To check this one

In [ ]:
def mean_reciprocal_rank(indices, labels):
    mrr_sum = 0.0
    for i, retrieved in enumerate(indices):
        for rank, idx in enumerate(retrieved):
            if labels[idx] == labels[i]:
                mrr_sum += 1.0 / (rank + 1)
                break
    return mrr_sum / len(indices)

def precision_at_k(indices, labels, k=100):
    precision_sum = 0.0
    for i, retrieved in enumerate(indices):
        relevant_retrieved = len([idx for idx in retrieved[:k] if labels[idx] == labels[i]])
        precision_sum += relevant_retrieved / k
    return precision_sum / len(indices)

def hit_rate_at_k(indices, labels, k=100):
    hit_rate_sum = 0.0
    for i, retrieved in enumerate(indices):
        if any(labels[idx] == labels[i] for idx in retrieved[:k]):
            hit_rate_sum += 1
    return hit_rate_sum / len(indices)

def text_to_image_retrieval(train_embeddings, train_labels, text_embeddings, k=100):

    nearest_neighbors_indices = []
    for query in text_embeddings:
        nearest_neighbors_indices.append(knn(train_embeddings, train_labels, query, k))
    
    mrr = mean_reciprocal_rank(nearest_neighbors_indices, train_labels)
    precision = precision_at_k(nearest_neighbors_indices, train_labels, k)
    hit_rate = hit_rate_at_k(nearest_neighbors_indices, train_labels, k)
    
    return mrr, precision, hit_rate

mrr, precision, hit_rate = text_to_image_retrieval(train_embeddings_cpu, train_labels_cpu, text_embeddings_cpu, k=100)

# Print the evaluation metrics
print(f"Mean Reciprocal Rank: {mrr}")
print(f"Precision@100: {precision}")
print(f"Hit Rate@100: {hit_rate}")

#### **Task-2.2 Retrieval**

In [ ]:
def image_to_image_retrieval(train_embeddings, train_labels, test_embeddings, test_labels, k=100):
    nearest_neighbors_indices = []
    total_comparisons = 0
    
    for query in test_embeddings:
        neighbors, comparisons = knn(train_embeddings, train_labels, query, k, return_comparisons=True)
        nearest_neighbors_indices.append(neighbors)
        total_comparisons += comparisons
    
    mrr = mean_reciprocal_rank(nearest_neighbors_indices, test_labels)
    precision = precision_at_k(nearest_neighbors_indices, test_labels, k)
    hit_rate = hit_rate_at_k(nearest_neighbors_indices, test_labels, k)
    
    avg_comparisons = total_comparisons / len(test_embeddings)
    
    return mrr, precision, hit_rate, avg_comparisons

mrr, precision, hit_rate, avg_comparisons = image_to_image_retrieval(train_embeddings_cpu, train_labels_cpu, test_embeddings_cpu, test_labels_cpu, k=100)

print(f"Mean Reciprocal Rank: {mrr}")
print(f"Precision@100: {precision}")
print(f"Hit Rate@100: {hit_rate}")
print(f"Average Comparisons per Query: {avg_comparisons}")

### **3.2**

#### **LSH Implementation**

In [ ]:
class LSH:
    def __init__(self, num_bits, num_planes, dimension):
       
        self.num_bits = num_bits
        self.num_planes = num_planes
        self.dimension = dimension
        
        self.hyperplanes = np.random.randn(self.num_planes, self.dimension)
        
    def hash(self, vector):

        dot_products = np.dot(self.hyperplanes, vector)
        hash_code = (dot_products > 0).astype(int)
        return hash_code
    
    def fit(self, dataset):
        
        self.hash_codes = np.array([self.hash(vector) for vector in dataset])
        
    def get_hash_buckets(self):
        
        hash_buckets = {}
        for code in self.hash_codes:
            code_tuple = tuple(code) 
            if code_tuple not in hash_buckets:
                hash_buckets[code_tuple] = 0
            hash_buckets[code_tuple] += 1
        return hash_buckets
    
    def search(self, query_vector, k=5):
        
        query_hash = self.hash(query_vector)
        distances = np.array([np.sum(query_hash != stored_hash) for stored_hash in self.hash_codes])
        nearest_indices = np.argsort(distances)[:k]
        return nearest_indices

def compute_metrics(train_embeddings, train_labels, test_embeddings, test_labels, k=5):
    
    lsh = LSH(num_bits=10, num_planes=50, dimension=train_embeddings.shape[1])
    lsh.fit(train_embeddings)
    
    mrr = 0
    precision_at_k = 0
    hit_rate = 0
    total_queries = len(test_embeddings)
    total_comparisons = 0 
    
    for query_embedding, query_label in zip(test_embeddings, test_labels):
        retrieved_indices = lsh.search(query_embedding, k)
        retrieved_labels = train_labels[retrieved_indices]
        
        rank = np.where(retrieved_labels == query_label)[0]
        if len(rank) > 0:
            mrr += 1 / (rank[0] + 1)
        
        precision_at_k += np.sum(retrieved_labels == query_label) / k
        
        if np.sum(retrieved_labels == query_label) > 0:
            hit_rate += 1
        
        total_comparisons += len(train_embeddings)
    
    mrr /= total_queries
    precision_at_k /= total_queries
    hit_rate /= total_queries
    
    avg_comparisons_per_query = total_comparisons / total_queries
    
    return mrr, precision_at_k, hit_rate, avg_comparisons_per_query

k = 100
mrr, precision_at_k, hit_rate, avg_comparisons_per_query = compute_metrics(train_embeddings_cpu, train_labels_cpu, test_embeddings_cpu, test_labels_cpu, k)

print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print(f"Precision@{k}: {precision_at_k:.4f}")
print(f"Hit Rate: {hit_rate:.4f}")
print(f"Average Comparisons Per Query: {avg_comparisons_per_query:.0f}")

### **3.3**

#### **IVF Implementation**

In [ ]:
class IVF:
    def __init__(self, embeddings, n_clusters=10):
        
        self.n_clusters = n_clusters
        self.embeddings = embeddings
        
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        self.kmeans.fit(embeddings)
        
        self.centroids = self.kmeans.cluster_centers_
        
        self.labels = self.kmeans.labels_

    def search(self, query_embedding, nprobe=1):
        query_cluster = self.kmeans.predict([query_embedding])[0]
        cluster_distances = pairwise_distances_argmin_min([query_embedding], self.centroids)[1]
        
        closest_clusters = np.argsort(cluster_distances)[:nprobe]  # Get the top nprobe clusters
        
        closest_indices = []
        for cluster in closest_clusters:
            cluster_indices = np.where(self.labels == cluster)[0]
            closest_indices.extend(cluster_indices)
        
        return closest_indices

def compute_metrics(train_embeddings, train_labels, test_embeddings, test_labels, ivf, k=100, nprobe=1):
    mrr = 0
    precision_at_k = 0
    hit_rate = 0
    total_queries = len(test_labels)
    total_comparisons = 0  

    for query_embedding, query_label in zip(test_embeddings, test_labels):
        retrieved_indices = ivf.search(query_embedding, nprobe)
        
        # Check if retrieved indices are valid
        if len(retrieved_indices) == 0:
            print(f"No results found for query {query_label}")
            continue
        
        retrieved_labels = train_labels[retrieved_indices]
        
        rank = np.where(retrieved_labels == query_label)[0]
        if len(rank) > 0:
            mrr += 1 / (rank[0] + 1)
        
        precision_at_k += np.sum(retrieved_labels == query_label) / k
        
        if np.sum(retrieved_labels == query_label) > 0:
            hit_rate += 1
        
        comparisons_for_query = 0
        for cluster in ivf.search(query_embedding, nprobe):
            comparisons_for_query += np.sum(ivf.labels == cluster)
        
        total_comparisons += comparisons_for_query
    
    mrr /= total_queries
    precision_at_k /= total_queries
    hit_rate /= total_queries
    avg_comparisons = total_comparisons / total_queries
    
    return mrr, precision_at_k, hit_rate, avg_comparisons


    
ivf = IVF(train_embeddings, n_clusters=10)

nprobe = 5 
mrr, precision, hit_rate, avg_comparisons = compute_metrics(train_embeddings_cpu, train_labels_cpu, test_embeddings_cpu, test_labels_cpu, ivf, k=100, nprobe=nprobe)
    
print(f"Mean Reciprocal Rank (MRR) @nprobe={nprobe}: {mrr:.4f}")
print(f"Precision@100 @nprobe={nprobe}: {precision:.4f}")
print(f"Hit Rate@100 @nprobe={nprobe}: {hit_rate:.4f}")
print(f"Average Comparisons per Query @nprobe={nprobe}: {avg_comparisons:.4f}")

#### **Plotting**

In [ ]:
def plot_cluster_sizes(ivf):
    unique_labels, counts = np.unique(ivf.labels, return_counts=True)
    plt.figure(figsize=(8, 6))
    plt.bar(unique_labels, counts, color='skyblue')
    plt.title('Number of Points in Each Cluster')
    plt.xlabel('Cluster ID')
    plt.ylabel('Number of Points')
    plt.show()

plot_cluster_sizes(ivf)

In [ ]:
def plot_comparisons_vs_nprobe(train_embeddings, train_labels, test_embeddings, test_labels, ivf, nprobes_range):
    avg_comparisons_list = []
    
    for nprobe in nprobes_range:
        mrr, precision, hit_rate, avg_comparisons = compute_metrics(train_embeddings, train_labels, test_embeddings, test_labels, ivf, k=100, nprobe=nprobe)
        avg_comparisons_list.append(avg_comparisons)
    
    plt.figure(figsize=(8, 6))
    plt.plot(nprobes_range, avg_comparisons_list, marker='o', color='orange')
    plt.title('Average Number of Comparisons per Query vs nprobe')
    plt.xlabel('nprobe')
    plt.ylabel('Average Comparisons')
    plt.grid(True)
    plt.show()

nprobes_range = [1, 2, 3, 5, 10, 20]
plot_comparisons_vs_nprobe(train_embeddings_cpu, train_labels_cpu, test_embeddings_cpu, test_labels_cpu, ivf, nprobes_range)